In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase.visualize import view
import xrayutilities as xu
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from tqdm import trange

In [2]:
#Making different background signals to be added to simualted data

glass_bkg=pd.read_excel('Copy of Glass slide.xls').to_numpy()
glass_bkg_np=[]
for i in range(0, len(glass_bkg)):
    glass_bkg_np.append((glass_bkg[i][0], glass_bkg[i][1]-932))

glass_bkg_np=np.array(glass_bkg_np)

zero_bkg=[]
for i in range(0, 9000):
    zero_bkg.append((np.arange(0, 90, 0.01)[i], 0))

zero_bkg=np.array(zero_bkg)

In [3]:
#Functions to simulate XRD patterns and save data to csvs

def make_mixed_XRD_pattern(mat_1, mat_2, ratio_1, ratio_2, I0, noise, noise_floor, background, gaussian, sigma):
    theta_start=np.min(background[:,0])
    theta_end=np.max(background[:,0])
    theta_step=(theta_end-theta_start)/len(background)
    tt = np.arange(theta_start, theta_end, theta_step)
    powder_1 = xu.simpack.Powder(mat_1, ratio_1)
    powder_2 = xu.simpack.Powder(mat_2, ratio_2)
    pm = xu.simpack.PowderModel(powder_1, powder_2, I0=I0)
    if gaussian==True:
        inte = pm.simulate(tt)
        inte=gaussian_filter1d(inte, sigma)+background[:,1]
        pm.close()
    if gaussian==False:
        inte = pm.simulate(tt)+background[:,1]
        pm.close()
    if noise==True:
        noise=np.random.normal(0,noise_floor, len(tt))
        inte=inte+noise
        
    return tt, inte, pm

def save_to_csv(data_list, material_A, material_B, savefolder):
    for i in range(0, len(data_list)):
        dataframe=pd.DataFrame({'2 theta':data_list[i][2][0], 'Intensity':data_list[i][2][1]})
        dataframe.to_csv(savefolder+'/'+f'{round(data_list[i][0]/(data_list[i][0]+data_list[i][1]), 2)}'+' '+ material_A+', '+f'{round(data_list[i][1]/(data_list[i][0]+data_list[i][1]), 2)}'+ material_A+'.csv')
    return dataframe

In [ ]:
#List of some materials in X-ray utlities materials library

material_list=[xu.materials.Ag2Se,
               xu.materials.Al2O3,
               xu.materials.BaF2,
               xu.materials.BaTiO3,
               xu.materials.CaF2,
               xu.materials.CaTiO3,
               xu.materials.CdSe,
               xu.materials.Co3O4,
               xu.materials.Fe3O4,
               xu.materials.FeO,
               xu.materials.LaB6,
               xu.materials.MgO,
               xu.materials.MnO,
               xu.materials.MnO2,
               xu.materials.NaCl,
               xu.materials.SiO2,
               xu.materials.SrTiO3,
               xu.materials.TiO2]

In [ ]:
#Making the data
#Change to effect ratios of different materials, only tested with integer values e.g. A=1, B=1 will create a 50:50 XRD pattern of materias A and B
A_numbers=[]
B_numbers=[]

#Example script using CaF2 and FeO
#Recommended I0=1e5 or higher, especially with glass background. Recommended sigma >3 to not eliminate shoulders but merge similar peaks (will depend on I0 value)
example_set=[]
for i in trange(0, len(A_numbers)):
    data=make_mixed_XRD_pattern(xu.materials.CaF2, xu.materials.FeO, ratio_1=A_numbers[i], ratio_2=B_numbers[i], I0=1e5, noise=True, noise_floor=0.05, background=zero_bkg, gaussian=False, sigma=0)
    example_set.append([A_numbers[i],B_numbers[i],data])

#save data to a csv folder, need to define savefolder
save_to_csv(example_set, 'CaF2', 'FeO', savefolder)